<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/project3_3113_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To remove punctuation and numbers
import re
import pandas as pd
import numpy as np
import warnings

#Vis
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
import statsmodels.api as sm

# Import BeautifulSoup for Removing HTML Markup
from bs4 import BeautifulSoup
# A stop word list from the Python Natural Language Toolkit (NLTK)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize



In [2]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##Goal
Your goal is to achieve an AUC score of at least 0.986 across all five test data splits.

## Resources:
- Kaggle: https://www.kaggle.com/c/word2vec-nlp-tutorial
- What we have tried 1: https://campuswire.com/c/GB46E5679/feed/785

In [4]:
DATA_DIR='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project3/Data/F24_Proj3_data'

## Part 1 By Zilal

#Logistic Regression


In [39]:
# Train the final model
def train_logistic(X_train, y_train):
    model = LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        l1_ratio=0, #means just Ridge Regression
        C=5
    )

    model.fit(X_train, y_train)
    return model

In [40]:
# Making Prediction
def predict(X_test):
    X_test = test.drop(columns=['id', 'review'])
    # Predict probabilities for the test data
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for positive sentiment
    return y_pred_proba


In [42]:
num_splits=5
for i in range(num_splits):
  train = pd.read_csv(f"{DATA_DIR}/split_{i+1}/train.csv")
  test = pd.read_csv(f"{DATA_DIR}/split_{i+1}/test.csv")


  # Separate features and target
  X_train = train.drop(columns=['id', 'sentiment', 'review'])
  y_train = train['sentiment']
  X_test = test.drop(columns=['id', 'review'])


  # Logistic regression without penalty as baseline
  model = train_logistic(X_train, y_train)
  y_pred_proba = predict(X_test)
  auc_baseline = roc_auc_score(y_test, y_pred_proba)
  print(f"Baseline Logistic Regression AUC in split {i+1}: {auc_baseline:.3f}")

  submission = pd.DataFrame({
    'id': test['id'],
    'prob': y_pred_proba
  })

  submission.to_csv(f'{DATA_DIR}/split_{i+1}/mysubmission.csv', index=False)

    # Calculate AUC on the test data
  y_test = pd.read_csv(f'{DATA_DIR}/split_{i+1}/test_y.csv')['sentiment']

  test_auc_score = roc_auc_score(y_test, y_pred_proba)
  print(f"Best AUC Score on Test Data: in split {i+1}", test_auc_score)


Baseline Logistic Regression AUC in split 1: 0.499
Best AUC Score on Test Data: in split 1 0.9871402088013582
Baseline Logistic Regression AUC in split 2: 0.501
Best AUC Score on Test Data: in split 2 0.9867615833393729
Baseline Logistic Regression AUC in split 3: 0.506
Best AUC Score on Test Data: in split 3 0.9864233090877216
Baseline Logistic Regression AUC in split 4: 0.504
Best AUC Score on Test Data: in split 4 0.986954014050569
Baseline Logistic Regression AUC in split 5: 0.503
Best AUC Score on Test Data: in split 5 0.9863475507543306


##**Part 2 By Jianci and Messay**

##Libraries And Packages


In [5]:
# Removing HTML Markup: The BeautifulSoup Package
!pip install BeautifulSoup4


##Data Cleaning and Text Preprocessing
- Removing HTML Markup: The BeautifulSoup Package
- Frequently occurring words that don't carry much meaning. Such words are called "stop words".

In [10]:

nltk.download('stopwords')  # Download text data sets, including stop words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    #
    # 2. Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    #
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    #
    # 6. Join the words back into one string separated by space,
    # and return the result.

    return( " ".join( meaningful_words ))

# Loop through and clean all of the training set at once

In [12]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size
print("Cleaning and parsing the training set movie reviews...\n")
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [13]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000,
                             ngram_range=(1, 4),            # Use 1- to 4-grams
                             min_df=0.001,                       # Minimum term frequency
                             max_df=0.5,                      # Maximum document frequency
                             token_pattern=r"\b[\w+|']+\b") # Use word tokenizer: See Ethan's comment below

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [14]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names_out()
print(vocab)

['abandoned' 'abilities' 'ability' ... 'zombie' 'zombies' 'zone']


In [ ]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

In [ ]:
print("Training the random forest...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the bag of words as
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


In [15]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()






Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000

